# Lab | Final regression model in "Health Care for All" Case

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', None)

In [3]:
targets = pd.read_csv('./files_for_lab/target.csv')
categorical = pd.read_csv('./files_for_lab/categorical.csv')
numerical = pd.read_csv('./files_for_lab/numerical.csv')

In [4]:
print(targets.shape)
print(categorical.shape)
print(numerical.shape)

(95412, 2)
(95412, 22)
(95412, 315)


In [5]:
print(targets.isna().sum().sum())
print(categorical.isna().sum().sum())
print(numerical.isna().sum().sum())

0
0
0


In [6]:
# targets

In [7]:
# categorical

In [8]:
# numerical

Income, wealth1 and wealth2 are categories but can be treated with the scaler as numerical.

### Concat.

In [9]:
df = pd.concat([categorical, numerical, targets], axis=1)
df.shape

(95412, 339)

### X-y split.

In [10]:
df = df[df['TARGET_B']==1]

In [11]:
df = df.drop('TARGET_B', axis=1)

In [12]:
X = df.drop('TARGET_D', axis=1)
y = df['TARGET_D']

In [13]:
print(X.shape)
y.shape

(4843, 337)


(4843,)

### Train-test split.

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

### Encoding.

In [15]:
X_train_num = X_train.select_dtypes(exclude=object)
X_train_cat = X_train.select_dtypes(include=object)

In [16]:
X_test_num = X_test.select_dtypes(exclude=object)
X_test_cat = X_test.select_dtypes(include=object)

In [17]:
# X_train_cat

Train.

In [18]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(drop='first', handle_unknown='ignore').fit(X_train_cat)

In [19]:
X_train_cat_encoded = encoder.transform(X_train_cat).toarray()

In [20]:
feature_names = encoder.get_feature_names_out(input_features=X_train_cat.columns)

X_train_cat_enc = pd.DataFrame(X_train_cat_encoded, index=X_train_cat.index, columns=feature_names)

Test.

In [21]:
X_test_cat_encoded = encoder.transform(X_test_cat).toarray()

In [22]:
feature_names = encoder.get_feature_names_out(input_features=X_test_cat.columns)

X_test_cat_enc = pd.DataFrame(X_test_cat_encoded, index=X_test_cat.index, columns=feature_names)

### Scaling.

Train.

In [23]:
from sklearn.preprocessing import MinMaxScaler
scaler= MinMaxScaler()

In [24]:
scaler.fit(X_train_num)

MinMaxScaler()

In [25]:
X_train_num_sc = pd.DataFrame(scaler.transform(X_train_num), index=X_train_num.index, columns=X_train_num.columns)

In [26]:
# X_train_num_sc

Test.

In [27]:
X_test_num_sc = pd.DataFrame(scaler.transform(X_test_num), index=X_test_num.index, columns=X_test_num.columns)

In [28]:
# X_test_num_sc

### Concat.

In [29]:
X_train_tr = pd.concat([X_train_num_sc, X_train_cat_enc], axis=1)

In [30]:
X_train_tr.isna().sum().sum()

0

In [31]:
X_test_tr = pd.concat([X_test_num_sc, X_test_cat_enc], axis=1)

In [32]:
X_test_tr.isna().sum().sum()

0

In [33]:
df_train_tr = pd.concat([X_train_tr, y_train], axis=1)

In [34]:
df_train_tr.isna().sum().sum()

0

In [35]:
df_test_tr = pd.concat([X_test_tr, y_test], axis=1)

In [36]:
df_test_tr.isna().sum().sum()

0

### Feature selection.

In a real-life scenario, we would do another feature selection for the linear regression model. This time, we will use the same feature selection we used for the classifcation model.

In [37]:
cols = pd.read_csv('final_cols_2.csv')

In [38]:
cols = cols['Column'].tolist()

In [39]:
X_train = X_train_tr[cols]
X_test = X_test_tr[cols]

In [40]:
X_train.isna().sum().sum()

0

In [41]:
X_test.isna().sum().sum()

0

### Correlation.

In [42]:
# corr_mat = X_train.corr()

# fig, ax = plt.subplots(figsize=(18,18))
# ax = sns.heatmap(corr_mat, annot=True)
# plt.show()

In [43]:
# We remove HV2, HVP2, HVP3, HVP6
# X_train = X_train.drop(['HV2', 'HVP2', 'HVP3', 'HVP6'], axis=1)

In [44]:
# corr_mat2 = X_train.corr()

# fig, ax = plt.subplots(figsize=(18,18))
# ax = sns.heatmap(corr_mat2, annot=True)
# plt.show()

In [45]:
# We remove HVP1, HVP4, RP1
# X_train = X_train.drop(['HVP1', 'HVP4', 'RP1'], axis=1)

In [46]:
# corr_mat3 = X_train.corr()

# fig, ax = plt.subplots(figsize=(18,18))
# ax = sns.heatmap(corr_mat3, annot=True)
# plt.show()

In [47]:
# X_test = X_test.drop(['HV2', 'HVP2', 'HVP3', 'HVP6', 'HVP1', 'HVP4', 'RP1'], axis=1)

In [48]:
# X_train

In [49]:
# X_test

### Linear regression model.

In [50]:
from sklearn import linear_model
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, r2_score

lm = linear_model.LinearRegression()

lm.fit(X_train, y_train)

cv_scores = cross_val_score(lm, X_train, y_train, cv=5, scoring='r2')
print("cross-validation r2 score: ", cv_scores.mean())

cross-validation r2 score:  0.3719965686406712


In [51]:
y_test_pred = lm.predict(X_test)

r2_test = r2_score(y_test, y_test_pred)
print("test r2 score: ", r2_test)

mse = mean_squared_error(y_test, y_test_pred)
rmse = np.sqrt(mse)
print("test MSE:", mse)
print("test RMSE:", rmse)

test r2 score:  0.38174405056984284
test MSE: 118.57973563097009
test RMSE: 10.889432291491145


### Predicting donation amount of previously predicted donors.

In [52]:
df_predB = pd.read_csv('df_predB_2.csv', index_col=0)

In [53]:
# df_predB

In [54]:
df = df_predB[df_predB['PRED_B']==1]

In [55]:
df.shape

(38620, 340)

In [56]:
df = df.drop(['TARGET_B', 'PRED_B'], axis=1)

In [57]:
X = df.drop('TARGET_D', axis=1)
y = df['TARGET_D']

In [58]:
X_cat= X.select_dtypes(include=object)
X_num= X.select_dtypes(exclude=object)

In [59]:
# X_cat

Encoding

In [60]:
X_cat_encoded = encoder.transform(X_cat).toarray()
feature_names = encoder.get_feature_names_out(input_features=X_cat.columns)
X_cat_enc = pd.DataFrame(X_cat_encoded, index=X_cat.index, columns=feature_names)

In [61]:
X_cat_enc

,STATE_FL,STATE_GA,STATE_IL,STATE_IN,STATE_MI,STATE_MO,STATE_NC,STATE_TX,STATE_WA,STATE_WI,STATE_other,HOMEOWNR_U,GENDER_M,GENDER_other,RFA_2A_E,RFA_2A_F,RFA_2A_G,GEOCODE2_B,GEOCODE2_C,GEOCODE2_D,DOMAIN_A_R,DOMAIN_A_S,DOMAIN_A_T,DOMAIN_A_U
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95402,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
95404,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
95406,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
95409,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


Scaling.

In [62]:
X_num_sc = pd.DataFrame(scaler.transform(X_num), index=X_num.index, columns=X_num.columns)

In [63]:
X_num_sc

,CLUSTER,DATASRCE,DOMAIN_B,ODATEW_YR,ODATEW_MM,DOB_YR,DOB_MM,MINRDATE_YR,MINRDATE_MM,MAXRDATE_YR,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_YR,FIRSTDATE_MM,TCODE,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,STATEGOV,FEDGOV,WEALTH2,POP901,POP902,POP903,POP90C1,POP90C2,POP90C3,POP90C4,POP90C5,ETH1,ETH2,ETH3,ETH4,ETH5,ETH6,ETH7,ETH8,ETH9,ETH10,ETH11,ETH12,ETH13,ETH14,ETH15,ETH16,AGE901,AGE902,AGE903,AGE904,AGE905,AGE906,AGE907,CHIL1,CHIL2,CHIL3,AGEC1,AGEC2,AGEC3,AGEC4,AGEC5,AGEC6,AGEC7,CHILC1,CHILC2,CHILC3,CHILC4,CHILC5,HHAGE1,HHAGE2,HHAGE3,HHN1,HHN2,HHN3,HHN4,HHN5,HHN6,MARR1,MARR2,MARR3,MARR4,HHP1,HHP2,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8,DW9,HV1,HV2,HV3,HV4,HU1,HU2,HU3,HU4,HU5,HHD1,HHD2,HHD3,HHD4,HHD5,HHD6,HHD7,HHD8,HHD9,HHD10,HHD11,HHD12,ETHC1,ETHC2,ETHC3,ETHC4,ETHC5,ETHC6,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR1,HUR2,RHP1,RHP2,RHP3,RHP4,HUPA1,HUPA2,HUPA3,HUPA4,HUPA5,HUPA6,HUPA7,RP1,RP2,RP3,RP4,MSA,ADI,DMA,IC1,IC2,IC3,IC4,IC5,IC6,IC7,IC8,IC9,IC10,IC11,IC12,IC13,IC14,IC15,IC16,IC17,IC18,IC19,IC20,IC21,IC22,IC23,HHAS1,HHAS2,HHAS3,HHAS4,MC1,MC2,MC3,TPE1,TPE2,TPE3,TPE4,TPE5,TPE6,TPE7,TPE8,TPE9,PEC1,PEC2,TPE10,TPE11,TPE12,TPE13,LFC1,LFC2,LFC3,LFC4,LFC5,LFC6,LFC7,LFC8,LFC9,LFC10,OCC1,OCC2,OCC3,OCC4,OCC5,OCC6,OCC7,OCC8,OCC9,OCC10,OCC11,OCC12,OCC13,EIC1,EIC2,EIC3,EIC4,EIC5,EIC6,EIC7,EIC8,EIC9,EIC10,EIC11,EIC12,EIC13,EIC14,EIC15,EIC16,OEDC1,OEDC2,OEDC3,OEDC4,OEDC5,OEDC6,OEDC7,EC1,EC2,EC3,EC4,EC5,EC6,EC7,EC8,SEC1,SEC2,SEC3,SEC4,SEC5,AFC1,AFC2,AFC3,AFC4,AFC5,AFC6,VC1,VC2,VC3,VC4,ANC1,ANC2,ANC3,ANC4,ANC5,ANC6,ANC7,ANC8,ANC9,ANC10,ANC11,ANC12,ANC13,ANC14,ANC15,POBC1,POBC2,LSC1,LSC2,LSC3,LSC4,VOC1,VOC2,VOC3,HC1,HC2,HC3,HC4,HC5,HC6,HC7,HC8,HC9,HC10,HC11,HC12,HC13,HC14,HC15,HC16,HC17,HC18,HC19,HC20,HC21,MHUC1,MHUC2,AC1,AC2,CARDPROM,NUMPROM,CARDPM12,NUMPRM12,RAMNTALL,NGIFTALL,CARDGIFT,MINRAMNT,MAXRAMNT,LASTGIFT,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2F,CLUSTER2
0,0.673077,1.0,0.333333,0.272727,0.0,0.397849,1.000000,0.545455,0.636364,0.785714,0.090909,0.0,1.000000,0.666667,0.909091,0.000000,0.595745,0.666667,1.000000,0.000000,0.000000,0.493671,0.343434,0.181818,0.208333,0.04,0.016667,0.555556,0.016206,0.017566,0.012278,0.0,0.353535,0.656566,0.474747,0.697368,0.929293,0.010101,0.000000,0.000000,0.113402,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.119565,0.0,0.0,0.000000,0.513158,0.631579,0.671053,0.540541,0.675676,0.729730,0.352113,0.313131,0.424242,0.272727,0.120879,0.141414,0.181818,0.500000,0.333333,0.211538,0.258621,0.121212,0.333333,0.507463,0.50,0.181818,0.273684,0.121951,0.247312,0.191489,0.388235,0.494949,0.282828,0.121212,0.040404,0.616162,0.070707,0.214286,0.206522,0.304615,0.394286,0.979798,0.959596,0.033333,0.020202,0.000000,0.000000,0.070707,0.098592,0.000000,0.080167,0.107155,0.230769,0.153846,0.868687,0.141414,0.969697,0.040816,0.070707,0.383838,0.808081,0.707071,0.323232,0.848485,0.161616,0.103448,0.2,0.089286,0.109756,0.200000,0.053571,0.239437,0.505051,0.260417,0.000000,0.000000,0.000000,0.020202,0.070707,0.131313,0.272727,0.474747,0.000000,0.011236,0.616162,0.690476,0.693182,0.245902,0.285714,0.023810,0.000000,0.000000,0.141414,0.014085,0.000000,0.000000,0.020202,0.050505,0.171717,0.737374,0.000000,0.274419,0.774120,0.204667,0.212000,0.250359,0.271162,0.098876,0.134021,0.433962,0.489362,0.389831,0.151515,0.018868,0.000000,0.000000,0.012195,0.040404,0.252525,0.400000,0.388060,0.171717,0.047619,0.00,0.000000,0.022989,0.294737,0.072727,0.525773,0.013889,0.464646,0.580645,0.031579,0.888889,0.080808,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.102564,0.013889,0.141304,0.155556,0.296296,0.04,0.454545,0.565657,0.646465,0.505051,0.646465,0.444444,0.626263,0.535354,1.000000,0.000000,0.000000,0.166667,0.058824,0.080808,0.194030,0.132353,0.00,0.06,0.121622,0.048387,0.312500,0.463415,0.10,0.16,0.044776,0.000000,0.053571,0.650794,0.016667,0.000000,0.14,0.228070,0.113208,0.106383,0.000000,0.121212,0.090909,0.055556,0.018519,0.081081,0.285714,0.04,0.016667,0.098592,0.787879,0.042553,0.000000,0.705882,0.161616,0.178571,0.58

Concat.

In [64]:
X = pd.concat([X_cat_enc, X_num_sc], axis=1)

In [65]:
X.isna().sum().sum()

0

Feature selection.

In [66]:
X = X[cols]

In [67]:
# X

In [68]:
# X = X.drop(['HV2', 'HVP2', 'HVP3', 'HVP6', 'HVP1', 'HVP4', 'RP1'], axis=1)

In [69]:
df_f = pd.concat([X,y], axis=1)
df_f.isna().sum().sum()

0

In [70]:
# df_f

In [71]:
# X

Model predictions.

In [72]:
predD = lm.predict(X)

In [73]:
df_predD = pd.DataFrame({'PRED_D': predD}, index=X.index)

In [74]:
# df_predB

In [75]:
df = df_predB[df_predB['PRED_B']==1]

In [76]:
df_pred = pd.concat([df, df_predD], axis=1)

In [90]:
# df_pred

These are the original figures of the mailing campaign (sending the mail to the whole list ot donors in the dataset).

In [82]:
print('Cost of mailing of', round(0.68 * len(df_predB), 2), '(', len(df_predB), 'mails)')

Cost of mailing of 64880.16 ( 95412 mails)


In [83]:
print('Total donation amount of', round(sum(df_predB['TARGET_D']), 2))
print('Total net profit of', round(sum(df_predB['TARGET_D']) - (0.68*len(df_predB)), 2))

Total donation amount of 75668.7
Total net profit of 10788.54


These would be the predictions of our models.

In [ ]:
print('Cost of mailing of', round(0.68 * sum(df_pred['PRED_B']), 2), '(', sum(df_pred['PRED_B']), 'mails)')

In [89]:
print('Total predicted donation amount of:', round(sum(df_pred['PRED_D']), 2))
print('Total predicted net profit of', round(sum(df_pred['PRED_D']) - (0.68 * sum(df_pred['PRED_B']))))

Total predicted donation amount of: 477067.63
Total predicted net profit of 450806


We have a really high predicted profit. This is due to the fact that our model over-predicts donors (the precision is really low, so we have a lot of false positives).

Now we focus on the individuals we predicted as donors that actually made a donation (true positives).

In [86]:
print('Cost of mailing of', round(0.68 * sum(df_pred['PRED_B']), 2), '(', sum(df_pred['PRED_B']), 'mails)')

Cost of mailing of 26261.6 ( 38620 mails)


In [87]:
donat_tp = df_pred[df_pred['TARGET_B']==1]['TARGET_D'].sum()
print('Total predicted donation amount (true positives) of', donat_tp)
print('Expected net profit using our models of', round(donat_tp - (0.68 * sum(df_pred['PRED_B'])), 2))

Total predicted donation amount (true positives) of 34395.17
Expected net profit using our models of 8133.57


The charity would be expending less on mailing cost, but the expected profit would be 2,5k less using our current models.